In [18]:
import requests
import cv2
import numpy as np
import os
import random

# Make sure your directories exist
if not os.path.exists('images'):
    os.makedirs('images')
    
if not os.path.exists('groundtruth'):
    os.makedirs('groundtruth')

def get_random_images_and_masks(latitude, longitude, iteration):
    zoom = 18
    # Increase the size to crop out watermark
    size = "500x500"
    maptype = "satellite"
    base_url = "https://maps.googleapis.com/maps/api/staticmap?"
    api_key = np.load("api_key.npy")[0]
    style = "&style=feature:all|element:all|visibility:off&style=feature:road|element:geometry|visibility:on|color:0xff0000"

    
    # Fetch styled map image
    url = base_url + "center={},{}&zoom={}&size={}&key={}".format(
        latitude, longitude, zoom, size, api_key
    ) + style

    response = requests.get(url)
    
    if response.status_code == 200:
        styled_map_img_name = "groundtruth_orig_{}.png".format(iteration)
        with open(styled_map_img_name, 'wb') as f:
            f.write(response.content)
    else:
        print("Error in getting the image", response.status_code)

    # Convert styled map to binary mask
    image = cv2.imread(styled_map_img_name)
    
    # Crop the image to original size and remove watermark
    cropped_image = image[50:450, 50:450]
    
    gray_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(gray_image, 1, 255, cv2.THRESH_BINARY)

    mask_img_name = 'groundtruth/boston_{}.png'.format(iteration)
    cv2.imwrite(mask_img_name, mask)
    os.remove(styled_map_img_name)  # remove the unprocessed road mask

    # Fetch satellite image
    url = base_url + "center={},{}&zoom={}&size={}&maptype={}&key={}".format(
        latitude, longitude, zoom, size, maptype, api_key
    )

    response = requests.get(url)
    
    if response.status_code == 200:
        satellite_img_name = "images/boston_{}.png".format(iteration)
        with open(satellite_img_name, 'wb') as f:
            f.write(response.content)
        
        # Read the image back in
        image = cv2.imread(satellite_img_name)
        
        # Crop the image to remove watermark
        cropped_image = image[50:450, 50:450]
        
        # Overwrite the original image with the cropped one
        cv2.imwrite(satellite_img_name, cropped_image)
    else:
        print("Error in getting the image", response.status_code)




# Initial coordinates for boston
latitude_min = 42.319725
latitude_max = 42.391438
longitude_min = -71.129176
longitude_max = -71.033732

i = 0
# Run the function to obtain all the images and masks within the specified coordinates
for latitude in np.arange(latitude_min, latitude_max, 0.003):
    for longitude in np.arange(longitude_min, longitude_max, 0.003):
        get_random_images_and_masks(latitude, longitude, i)
        i += 1
        if i % 100 == 0:
            print(i, "images downloaded")
    #     if i > 2:
    #         break
    # if i > 2:
    #     break

In [8]:
# Initial coordinates for los angeles
latitude_min = 34.031936
latitude_max = 34.097919
longitude_min = -118.380975
longitude_max = -118.273171

i = 0
# Run the function to obtain all the images and masks within the specified coordinates
for latitude in np.arange(latitude_min, latitude_max, 0.003):
    for longitude in np.arange(longitude_min, longitude_max, 0.003):
        get_random_images_and_masks(latitude, longitude, i)
        i += 1
        if i % 100 == 0:
            print(i, "images downloaded")


100 images downloaded
200 images downloaded
300 images downloaded
400 images downloaded
500 images downloaded
600 images downloaded
700 images downloaded
